In [17]:
%load_ext autoreload
%autoreload 2
from geolifeclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
data_root = "/mnt/data/geolifeclef-2024/data"
edges_root = f"{data_root}/processed/geolsh_graph/v1/edges/threshold=100000"
print(edges_root)
edges = spark.read.parquet(edges_root)
edges.printSchema()
edges.show(5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/mnt/data/geolifeclef-2024/data/processed/geolsh_graph/v1/edges/threshold=100000


24/05/19 15:28:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- srcDataset: string (nullable = true)
 |-- srcSurveyId: integer (nullable = true)
 |-- srcSpeciesId: double (nullable = true)
 |-- dstDataset: string (nullable = true)
 |-- dstSurveyId: integer (nullable = true)
 |-- dstSpeciesId: double (nullable = true)
 |-- euclidean: double (nullable = true)

+----------+-----------+------------+----------+-----------+------------+------------------+
|srcDataset|srcSurveyId|srcSpeciesId|dstDataset|dstSurveyId|dstSpeciesId|         euclidean|
+----------+-----------+------------+----------+-----------+------------+------------------+
|   pa_test|       1792|        NULL|        po|      58960|      5790.0| 33971.22140053323|
|   pa_test|       4889|        NULL|  pa_train|    2679616|      9465.0| 37433.69463964683|
|   pa_test|       4889|        NULL|        po|     652126|      3969.0|26198.718375295055|
|   pa_test|       4889|        NULL|        po|    2094593|      4898.0|31014.268319388488|
|   pa_test|       8956|        NULL|     

In [10]:
edges.count()

1116195844

In [19]:
filtered = (
    edges.where("srcDataset = 'po'")
    .where("dstDataset = 'po'")
    .selectExpr("srcSurveyId as src", "dstSurveyId as dst")
    .distinct()
    .where("src != dst")
    .sample(0.01, seed=42)
    .limit(int(1e6))
)
# count unique surveys
filtered.selectExpr("src as survey").union(
    filtered.selectExpr("dst as survey")
).distinct().count()

24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/19 15:49:21 WARN RowBasedKeyValueBatch: Calling spill() on

1233465